# Introduction to Conjoint Analysis

Product management is all about trade-offs. Whether the objective is increased market share, profit margin or revenue, every product manager makes trade-offs—quality vs. cost, time to market vs. breadth of features, richness of the offering vs. ease of use, etc.

So, how do you know what the market wants? What market segments exist? What those segments prefer? What will they pay? In short, how do you know what trade-offs to make? The answer is to get the market to make the trade-offs for you. Not the entire market, of course, just a representative sample of the market.

By using conjoint analysis, you, as a product manager, can do just that: understand the trade-offs you should make by understanding the trade-offs your market will make. Then, apply your increased market insight to your revenue, profit or share objective.

Conjoint analysis is a set of market research techniques that measures the value the market places on each feature of your product and predicts the value of any combination of features. Conjoint analysis is, at its essence, all about features and trade-offs.

In [1]:
import pandas as pd  
import numpy as np  

import statsmodels.api as sm  
import statsmodels.formula.api as smf  

from patsy.contrasts import Sum

**Sum coding** compares the mean of the dependent variable for a given level to the overall mean of the dependent variable over all the levels. That is, it uses contrasts between each of the first ```k-1``` levels and level ```k``` In this example, level 1 is compared to all the others, level 2 to all the others, and level 3 to all the others.

In [2]:
# read in conjoint survey profiles with respondent ranks
conjoint_data_frame = pd.read_csv('mobile_services_ranking.csv')
conjoint_data_frame

,brand,startup,monthly,service,retail,apple,samsung,google,ranking
0,"""AT&T""","""$100""","""$100""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung NO""","""Nexus NO""",11
1,"""Verizon""","""$300""","""$100""","""4G NO""","""Retail YES""","""Apple YES""","""Samsung YES""","""Nexus NO""",12
2,"""US Cellular""","""$400""","""$200""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung YES""","""Nexus NO""",9
3,"""Verizon""","""$400""","""$400""","""4G YES""","""Retail YES""","""Apple NO""","""Samsung NO""","""Nexus NO""",2
4,"""Verizon""","""$200""","""$300""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung YES""","""Nexus YES""",8
5,"""Verizon""","""$100""","""$200""","""4G YES""","""Retail NO""","""Apple YES""","""Samsung NO""","""Nexus YES""",13
6,"""US Cellular""","""$300""","""$300""","""4G YES""","""Retail NO""","""Apple YES""","""Samsung NO""","""Nexus NO""",7
7,"""AT&T""","""$400""","""$300""","""4G NO""","""Retail YES""","""Apple YES""","""Samsung NO""","""Nexus YES""",4
8,"""AT&T""","""$200""","""$400""","""4G YES""","""Retail NO""","""Apple YES""","""Samsung YES""","""Nexus NO""",5
9,"""T-Mobile""","""$400""","""$100""","""4G YES""","""Retail NO""","""Apple YES""","""Samsung YES""","""Nexus YES""",16


* When doing conjoint analysis, we use **sum contrasts**, so that the sum of the fitted regression coefficients across the levels of each attribute is zero. 
* The fitted regression coefficients represent conjoint measures of utility called **part-worths**. 
> Part-worths reflect the strength of individual consumer preferences for each level of each attribute in the study.
> * Positive part-worths add to a product's value in the mind of the consumer.
> * Negative part-worths subtract from the product's value in the mind of the consumer.
* When we sum across the part-worths of a product, we obtain a measure of the utility (or benefit) to the consumer.
> **Utility**: The subjective preference judgement of an individual that represent the total value or worth that s/he is putting on the product having a combination of certain attributes.


In [3]:
# set up sum contrasts for effects coding as needed for conjoint analysis
# using C(effect, Sum) notation within main effects model specification
main_effects_model = 'ranking ~ C(brand, Sum) + C(startup, Sum) +  \
    C(monthly, Sum) + C(service, Sum) + C(retail, Sum) + C(apple, Sum) + \
    C(samsung, Sum) + C(google, Sum)'

In [4]:
# fit linear regression model using main effects only (no interaction terms)
main_effects_model_fit = smf.ols(main_effects_model, data = conjoint_data_frame).fit()
print main_effects_model_fit.summary() 

                            OLS Regression Results                            
Dep. Variable:                ranking   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     97.07
Date:                Fri, 29 Sep 2017   Prob (F-statistic):             0.0794
Time:                        14:39:35   Log-Likelihood:                 10.568
No. Observations:                  16   AIC:                             8.864
Df Residuals:                       1   BIC:                             20.45
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Intercept 

/Users/weekiang/anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


In [5]:
conjoint_attributes = ['brand', 'startup', 'monthly', 'service', 'retail', 'apple', 'samsung', 'google']

In [6]:
# build part-worth information one attribute at a time
level_name = []
part_worth = []
part_worth_range = []
end = 1  # initialize index for coefficient in params
for item in conjoint_attributes:
    nlevels = len(list(np.unique(conjoint_data_frame[item])))
    level_name.append(list(np.unique(conjoint_data_frame[item]))) 
    begin = end 
    end = begin + nlevels - 1
    new_part_worth = list(main_effects_model_fit.params[begin:end])
    new_part_worth.append((-1) * sum(new_part_worth))  
    part_worth_range.append(max(new_part_worth) - min(new_part_worth))  
    part_worth.append(new_part_worth)   
    # end set to begin next iteration
    

In [7]:
# compute attribute relative importance values from ranges
attribute_importance = []
for item in part_worth_range:
    attribute_importance.append(round(100 * (item / sum(part_worth_range)),2))

print attribute_importance

[2.38, 7.14, 51.19, 16.67, 2.38, 2.38, 10.71, 7.14]


In [8]:
# user-defined dictionary for printing descriptive attribute names     
effect_name_dict = {'brand' : 'Mobile Service Provider', \
    'startup' : 'Start-up Cost', 'monthly' : 'Monthly Cost', \
    'service' : 'Offers 4G Service', 'retail' : 'Has Nearby Retail Store', \
    'apple' : 'Sells Apple Products', 'samsung' : 'Sells Samsung Products', \
    'google' : 'Sells Google/Nexus Products'}  
 

In [9]:
# report conjoint measures 
index = 0  # initialize for use in for-loop
for item in conjoint_attributes:
    print('\nAttribute:', effect_name_dict[item])
    print('    Importance:', attribute_importance[index])
    print('    Level Part-Worths')
    for level in range(len(level_name[index])):
        print('       ',level_name[index][level], part_worth[index][level])       
    index = index + 1

('\nAttribute:', 'Mobile Service Provider')
('    Importance:', 2.38)
    Level Part-Worths
('       ', '"AT&T"', 6.8833827526759706e-15)
('       ', '"T-Mobile"', -0.25000000000001399)
('       ', '"US Cellular"', 3.7747582837255322e-15)
('       ', '"Verizon"', 0.25000000000000333)
('\nAttribute:', 'Start-up Cost')
('    Importance:', 7.14)
    Level Part-Worths
('       ', '"$100"', 0.75000000000001288)
('       ', '"$200"', 1.3988810110276972e-14)
('       ', '"$300"', -2.6645352591003757e-14)
('       ', '"$400"', -0.75000000000000022)
('\nAttribute:', 'Monthly Cost')
('    Importance:', 51.19)
    Level Part-Worths
('       ', '"$100"', 4.9999999999999964)
('       ', '"$200"', 1.9999999999999947)
('       ', '"$300"', -1.24999999999999)
('       ', '"$400"', -5.7500000000000009)
('\nAttribute:', 'Offers 4G Service')
('    Importance:', 16.67)
    Level Part-Worths
('       ', '"4G NO"', -1.7499999999999993)
('       ', '"4G YES"', 1.7499999999999993)
('\nAttribute:', 'Has Nearby

![Alt](spine_mobile.png "Spine chart")